Нужно установить weaviate-client

In [36]:
import weaviate
from weaviate.classes.init import Auth
from weaviate.classes.config import Property, DataType


client = weaviate.connect_to_local(
    host="cat-vm2.v6.rocks",  # Use a string to specify the host
    port=8080,
    auth_credentials=Auth.api_key("Hack_the_VK"),
    skip_init_checks=True,
)

assert client.is_ready()

In [2]:
print(client.is_ready())

True


In [ ]:
from weaviate.classes.config import Configure, Property, DataType

try:
# Note that you can use `client.collections.create_from_dict()` to create a collection from a v3-client-style JSON object
    client.collections.create(
        "Articles",
        properties=[
            Property(name="title", data_type=DataType.TEXT),
            Property(name="body", data_type=DataType.TEXT),
        ],
        vectorizer_config=[
            Configure.NamedVectors.text2vec_ollama(
                name="common_vectorizer",
                source_properties=["title", "body"],
                api_endpoint="http://ollama:11434",
            )
        ]
    )
except Exception as e:
    print(e)



/home/rin/.cache/pypoetry/virtualenvs/cat-spider-3u5woAeV-py3.12/lib/python3.12/site-packages/weaviate/collections/classes/config.py:2082: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  for cls_field in self.model_fields:


In [ ]:
collection = client.collections.get("red")


In [48]:
client.collections.delete("Articles")

In [45]:
existing_classes = client.schema.get()
existing_classes

AttributeError: 'WeaviateClient' object has no attribute 'schema'

In [49]:
# def create_collection(client: weaviate.WeaviateClient, model: str, name: str):
client.collections.create(
    "Articles",
    vectorizer_config=[
        Configure.NamedVectors.text2vec_ollama(
            name="title_vector",
            source_properties=["title"],
            api_endpoint="http://ollama:11434",  
            model="mistral"  
        ),
        Configure.NamedVectors.text2vec_ollama(
            name="content_vector",
            source_properties=["chunk"],  # each doc has chunks
            api_endpoint="http://ollama:11434",  
            model="mistral"
        )
    ],
)


In [ ]:
create_collection(client, "mistral", "red")

In [52]:
collection = client.collections.get("Articles")

for item in collection.iterator():
    print(item.uuid, item.properties)

e42a2d65-38f4-4372-9330-e91922f9aa72 {'title': 'The Travels of Marco Polo', 'body': "A 13th-century travelogue describing Polo's travels"}


In [51]:
res = collection.data.insert({
    "title": "The Travels of Marco Polo",
    "body": "A 13th-century travelogue describing Polo's travels",
})

In [73]:
client.close()